# 06 SemanticKernel | 04 Memory

## Memory

With Memories Semantic Kernel provides an high level abstraction which allows storing and querying text information using Embeddings and Vector Databases.

Additional information: 
- [Overview Memory](./README.md)
- [What is Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/overview/)

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Memory Builder instance

Similar to an instance of Semantic Kernel a Memory instance can be created using the MemoryBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with memory related functions. It abstracts e.g., embedding models and can communicate with Vector Databases for storing and retrieving vectors.

In the sample a volatile memory store is used (`VolatileMemoryStore()`) which can be used for development. For production scenarios external vector databases should be used. Here's a [list](https://learn.microsoft.com/en-us/semantic-kernel/memories/vector-db) of currently available Semantic Kernel connectors to vector databases.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory; 
using Microsoft.SemanticKernel.Plugins.Memory; 
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("SKIT_EMBEDDING_DEPLOYMENTNAME");

ISemanticTextMemory memoryBuilder = new MemoryBuilder()
    .WithAzureTextEmbeddingGenerationService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: embeddingDeploymentName)
    .WithMemoryStore(new VolatileMemoryStore())
    .Build();

Console.WriteLine($"Memory Builder created...");


#### Expected output:

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.SemanticKernel, 1.0.0-beta1
Memory Builder created.....
```

## Step 2: Define Knowledge

A definition of arbitrary facts which will be stored and queried using Semantic Kernel Memories.

In [ ]:
Dictionary<string,string> facts = new Dictionary<string,string>();

facts.Add(
    "1; Azure Container Instances; https://docs.microsoft.com/en-us/azure/container-instances/", 
        @"Containers are becoming the preferred way to package, deploy, and manage cloud applications. 
        Azure Container Instances offers the fastest and simplest way to run a container in Azure, 
        without having to manage any virtual machines and without having to adopt a higher-level service.
        Azure Container Instances is a great solution for any scenario that can operate in isolated containers, 
        including simple applications, task automation, and build jobs. For scenarios where you need full container orchestration, 
        including service discovery across multiple containers, automatic scaling, and coordinated application upgrades.
        We recommend reading through the considerations and limitations and the FAQs 
        to understand the best practices when deploying container instances."
);

facts.Add(
    "1; Azure Machine Learning; https://docs.microsoft.com/en-us/azure/machine-learning/", 
        @"Azure Machine Learning is a cloud service for accelerating and managing the machine learning project lifecycle. 
        Machine learning professionals, data scientists, and engineers can use it in their day-to-day workflows: 
        Train and deploy models, and manage MLOps.
        You can create a model in Azure Machine Learning or use a model built from an open-source platform, 
        such as Pytorch, TensorFlow, or scikit-learn. MLOps tools help you monitor, retrain, and redeploy models."
);

facts.Add(
    "2; Azure SQL Service; https://docs.microsoft.com/en-us/azure/azure-sql/", 
        @"Azure SQL is a family of managed, secure, and intelligent products that use the SQL Server database 
        engine in the Azure cloud. Azure SQL is built upon the familiar SQL Server engine, so you can migrate 
        applications with ease and continue to use the tools, languages, and resources you're familiar with. 
        Your skills and experience transfer to the cloud, so you can do even more with what you already have"
);

Console.WriteLine($"Facts loaded...");


#### Expected output:

```
Facts loaded...
```

## Step 4:Store facts

The instance of `MemoryBuilder` provides an function `SaveReferenceAsync()` which can be used to store text information with additional attributes like description, externalId, externalSourceName etc. MemoryBuilder performs behind the scenes calls to the registered Embeddings LLM to convert the provided textual information into a vector representation and stores the vector in the provided vector database. 

In [ ]:
string memoryCollectionName = "SummarizedAzureDocs";

foreach (var fact in facts) {

    await memoryBuilder.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: fact.Key.Split(";")[1].Trim(),
        text: fact.Value,
        externalId: fact.Key.Split(";")[2].Trim(),
        externalSourceName: "Azure Documenation"
    );
    
    Console.WriteLine($"Summarizations reference saved: '{fact.Key.Split(";")[1].Trim()}' ...");
}


#### Expected output:

```
Summarizations reference saved: 'Azure Container Instances' ...
Summarizations reference saved: 'Azure Machine Learning' ...
Summarizations reference saved: 'Azure SQL Service' ...
```

### Step 5 - Query Memory

Querying memories is similarly simple like storing information. The function `SearchAsync()` provided by the MemoryBuilder instance performs the necessary tasks to convert the query text into a vector representation using an Embedding LLM as well as communicating with the registered vector db to query content based on the query.

In [ ]:
string[] queryPhrases = {
    "Data Storage", 
    "Kubernetes",
    "Artificial Intelligence"
};

foreach (string query in queryPhrases) {
    
    IAsyncEnumerable<MemoryQueryResult> memoryQueryResults = memoryBuilder.SearchAsync(
                collection: memoryCollectionName, 
                query: query, 
                limit:1, 
                minRelevanceScore: 0.75);
    
    await foreach (MemoryQueryResult memoryQueryResult in memoryQueryResults)
    {
        Console.WriteLine($"Query: {query}");
        Console.WriteLine($"  Id:         : {memoryQueryResult.Metadata.Id}");
        Console.WriteLine($"  Description : {memoryQueryResult.Metadata.Description}");
        Console.WriteLine($"  Relevance   : {memoryQueryResult.Relevance}");
    }
}


Expected Outcome

```
Query: Data Storage
  Id:         : https://docs.microsoft.com/en-us/azure/azure-sql/
  Description : Azure SQL Service
  Relevance   : 0.7799396182360798
Query: Kubernetes
  Id:         : https://docs.microsoft.com/en-us/azure/container-instances/
  Description : Azure Container Instances
  Relevance   : 0.8090541118690212
Query: Artificial Intelligence
  Id:         : https://docs.microsoft.com/en-us/azure/machine-learning/
  Description : Azure Machine Learning
  Relevance   : 0.7889358688189336
```

## Next steps

- [Use Semantic Kernel Planner](./05_Planner.ipynb)

